# Report

Firstly, I know this paper is not new, and does not meet the qualification of 2019 or newer, but I feel as if this paper is so fundamental in Nurual collaborative filtering I felt that it would be good to take a look at it.

### Brief overview

Nural collaborative filtering is a way to recommend content to users based on their past interactions, and what other similar users have interacted with. It combines the results of Generalized Matrix Factorization (GMF) and Multi-Layer Perceptron (MLP) to learn and make predictions.

### Changes made

With these models being older, the code did not run on Pythin 3.10.16 in a andacond3 environment. I made changes to each of the model in order to make them runable.

### Scientific contribution/messing around

Both datasets used in the report (MovieLens and Pinterest) are large datasets which go off of a very specific metric. MovieLens has user ratings to key off of, and pinterest has list of users pins. I want to test the model on a seperate dataset which and see how well it performs, and if I can see the same results of increasing Hit Ratio (HR) from our Nural Collaborative Filter VS GMF or MLP.

I have chosen board game reviews. The entire dataset can be found here

https://www.kaggle.com/code/simonebergmann/collaborative-filter-for-boardgames





In [11]:
import numpy as np
import pandas as pd
from keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))



df_ratings = pd.read_csv("Data/board_games_data/user_ratings.csv")
print(df_ratings.info())

GPU Available: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18942215 entries, 0 to 18942214
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 433.6+ MB
None


In [12]:
# Given the size of the databse lets take a smaller sample to save our computer
df_ratings = df_ratings.sample(frac=0.001, random_state=42)
print(f"Sampled dataset size: {len(df_ratings)} rows")

from sklearn.model_selection import train_test_split

# Map users to integers
user_mapping = {user: idx for idx, user in enumerate(df_ratings['Username'].unique())}
df_ratings['UserId'] = df_ratings['Username'].map(user_mapping)
df_ratings = df_ratings.drop(columns=['Username'])

print(df_ratings.head())
print(df_ratings.info())


Sampled dataset size: 18942 rows
           BGGId  Rating  UserId
16597337   17709     2.0       0
15421575  155693     2.0       1
16051321   40531     7.0       2
16392352   22864     7.0       3
5484430       50     8.0       4
<class 'pandas.core.frame.DataFrame'>
Index: 18942 entries, 16597337 to 15579506
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   BGGId   18942 non-null  int64  
 1   Rating  18942 non-null  float64
 2   UserId  18942 non-null  int64  
dtypes: float64(1), int64(2)
memory usage: 591.9 KB
None


In [13]:

df_ratings = df_ratings.groupby(['UserId', 'BGGId'], as_index=False).mean()

train, test = train_test_split(df_ratings, test_size=0.2, random_state=42)

# Reindex UserId and BGGId in train and test to ensure zero-indexed and sequential values
user_mapping = {user: idx for idx, user in enumerate(pd.concat([train['UserId'], test['UserId']]).unique())}
item_mapping = {item: idx for idx, item in enumerate(pd.concat([train['BGGId'], test['BGGId']]).unique())}

train['UserId'] = train['UserId'].map(user_mapping)
train['BGGId'] = train['BGGId'].map(item_mapping)
test['UserId'] = test['UserId'].map(user_mapping)
test['BGGId'] = test['BGGId'].map(item_mapping)

# Import GMF Model
from GMF import get_model, parse_args
from GMF import get_train_instances
from GMF import evaluate_model
from scipy.sparse import dok_matrix
from CustomDataset import CustomDataset

# Get the number of unique users and items
num_users = len(user_mapping)
num_items = len(item_mapping)

# Define model parameters
mf_dim = 8  # Embedding size for MF
reg_mf = [0, 0]  # Regularization for MF embeddings

# Initialize GMF model
gmf_model = get_model(num_users, num_items, mf_dim, reg_mf)
gmf_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Vectorize the training data
train['UserId'] = pd.factorize(train['UserId'])[0]
train['BGGId'] = pd.factorize(train['BGGId'])[0]

# Create dataset
dataset = CustomDataset(train, test, num_users, num_items)
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives

# Generate training instances
num_negatives = 4  # Number of negative samples
user_input, item_input, labels = get_train_instances(train, num_negatives)

# Train the model
gmf_model.fit([np.array(user_input), np.array(item_input)],  # Inputs
          np.array(labels),  # Labels
          batch_size=256,
          epochs=1,
          verbose=1)

# Prepare test data
test_ratings = test[['UserId', 'BGGId', 'Rating']].values
# Generate negative samples for testing
test_negatives = []
all_items = set(range(num_items))  # All possible item IDs
for _, row in test.iterrows():
    user = row['UserId']
    positive_item = row['BGGId']
    # Exclude items the user interacted with in the training set
    interacted_items = {i for _, i in train.keys() if _ == user}
    negative_items = list(all_items - interacted_items)
    negative_samples = np.random.choice(negative_items, size=99, replace=False).tolist()  # 99 negatives
    test_negatives.append(negative_samples)

# Evaluate the model
hits, ndcgs = evaluate_model(gmf_model, test_ratings, test_negatives, 10, 1)
print(f"HR: {np.mean(hits)}, NDCG: {np.mean(ndcgs)}")

/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['user_input', 'item_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7985 - loss: 0.6727


/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['user_input', 'item_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


HR: 0.020849828450778568, NDCG: 0.010082173126259572


In [14]:
# Run MLP
from MLP import get_model, parse_args
from MLP import get_train_instances
from MLP import evaluate_model

# Define model parameters
layers = [64,32,16,8]  # Embedding size for MF
reg_layers = [0,0,0,0]  # Regularization for MF embeddings

# Initialize MLP model
# def get_model(num_users, num_items, layers=[20, 10], reg_layers=[0, 0]):
mlp_model = get_model(num_users, num_items, layers, reg_layers)
mlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the MLP model
mlp_model.fit([np.array(user_input), np.array(item_input)],  # Inputs
          np.array(labels),  # Labels
          batch_size=256,
          epochs=1,
          verbose=1)
# Evaluate the MLP model
hits, ndcgs = evaluate_model(mlp_model, test_ratings, test_negatives, 10, 1)
print(f"HR: {np.mean(hits)}, NDCG: {np.mean(ndcgs)}")

/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


296/296 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7719 - loss: 0.5804
HR: 0.46582211665347056, NDCG: 0.27873473940724885


In [15]:
# Run NeuMF
from NeuMF import get_model, parse_args
from NeuMF import get_train_instances
from NeuMF import evaluate_model

# Define model parameters
layers = [64,32,16,8]
reg_layers = [0,0,0,0]
nm_factors = 8
mf_dim = 8
reg_mf = 0

# def get_model(num_users, num_items, mf_dim=10, layers=[10], reg_layers=[0], reg_mf=0):
# Initialize NeuMF model
neumf_model = get_model(num_users, num_items, mf_dim, layers, reg_layers, reg_mf)
neumf_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the NeuMF model
neumf_model.fit([np.array(user_input), np.array(item_input)],  # Inputs
          np.array(labels),  # Labels
          batch_size=256,
          epochs=1,
          verbose=1)
# Evaluate the NeuMF model
hits, ndcgs = evaluate_model(neumf_model, test_ratings, test_negatives, 10, 1)
print(f"HR: {np.mean(hits)}, NDCG: {np.mean(ndcgs)}")

296/296 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7982 - loss: 0.5741
HR: 0.4745315386645553, NDCG: 0.2857013866550117


## Conclusion

From using a different dataset we see the same results that the neural collaborative filtering in provides a better HR % than MLF or GMF. This gives us a very powerful tool that we can use for content recomendation engines